In [14]:
import numpy as np
import pandas as pd
import imageio.v3 as imageio
import albumentations as A

from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch import nn
import torchvision.transforms.functional as F
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import os
import random
import joblib

tqdm.pandas()

In [15]:
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
DATA_DIRECTORY = os.path.join(os.getcwd(),"data")
torch.cuda.empty_cache()
# torch.cuda.set_per_process_memory_fraction(0.99, device=0)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

In [16]:
class Config():
    IMAGE_SIZE = 384
    TARGET_COLUMNS = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
    N_TARGETS = len(TARGET_COLUMNS)
    BATCH_SIZE = 10
    LR_MAX = 1e-4
    WEIGHT_DECAY = 0.01
    N_EPOCHS = 6
    TRAIN_MODEL = True
    IS_INTERACTIVE =False # os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
        
CONFIG = Config()

In [17]:
import os
path =  os.path.join(os.getcwd() , 'data')
imgs_train = f"{path}/train_images"
imgs_test = f"{path}/test_images"

train = pd.read_csv(f"{path}/train.csv")
train['file_path'] = train['id'].apply(lambda s: f'{path}/train_images/{s}.jpeg')
train['jpeg_bytes'] = train['file_path'].apply(lambda fp: open(fp, 'rb').read())
train.to_pickle('train.pkl')

for column in CONFIG.TARGET_COLUMNS:
    lower_quantile = train[column].quantile(0.005)
    upper_quantile = train[column].quantile(0.985)  
    train = train[(train[column] >= lower_quantile) & (train[column] <= upper_quantile)]

CONFIG.N_TRAIN_SAMPLES = len(train)
CONFIG.N_STEPS_PER_EPOCH = (CONFIG.N_TRAIN_SAMPLES // CONFIG.BATCH_SIZE)
CONFIG.N_STEPS = CONFIG.N_STEPS_PER_EPOCH * CONFIG.N_EPOCHS + 1

test = pd.read_csv(f"{path}/test.csv")
test['file_path'] = test['id'].apply(lambda s: f'{path}/test_images/{s}.jpeg')
test['jpeg_bytes'] = test['file_path'].apply(lambda fp: open(fp, 'rb').read())
test.to_pickle('test.pkl')

In [18]:
LOG_FEATURES = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

y_train = np.zeros_like(train[CONFIG.TARGET_COLUMNS], dtype=np.float32)
for target_idx, target in enumerate(CONFIG.TARGET_COLUMNS):
    v = train[target].values
    if target in LOG_FEATURES:
        v = np.log10(v)
    y_train[:, target_idx] = v

SCALER = StandardScaler()
y_train = SCALER.fit_transform(y_train)

In [19]:
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

TEST_TRANSFORMS = A.Compose([
        A.Resize(CONFIG.IMAGE_SIZE, CONFIG.IMAGE_SIZE),
        A.ToFloat(),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1),
        ToTensorV2(),
    ])

class Dataset(Dataset):
    def __init__(self, X_jpeg_bytes, y, transforms=None):
        self.X_jpeg_bytes = X_jpeg_bytes
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.X_jpeg_bytes)

    def __getitem__(self, index):
        X_sample = self.transforms(
            image=imageio.imread(self.X_jpeg_bytes[index]),
        )['image']
        y_sample = self.y[index]
        
        return X_sample, y_sample

test_dataset = Dataset(
    test['jpeg_bytes'].values,
    test['id'].values,
    TEST_TRANSFORMS,
)

In [20]:
BASE_DIR = os.path.join(os.getcwd() , 'data')
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = torch.load(f'{BASE_DIR}/model_08_ensemble.pth', map_location='cuda')
        
    def forward(self, inputs):
        return self.backbone(inputs)

model = Model()
model = model.to('cuda')

/tmp/ipykernel_79770/1456432996.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.backbone = torch.load(f'{BASE_DIR}/model_08_ensemble.pth', map_location='cuda')


In [21]:
def tta_augmentations(image):
    aug_images = []
    # Original image
    aug_images.append(image)
    # Flipped images
    aug_images.append(F.vflip(image))  # Vertical flip
    aug_images.append(F.hflip(image))  # Horizontal flip
    # Rotated images
    angle = random.uniform(-30, 30)  # Random rotation angle between -30 and 30 degrees
    rotated_image = F.rotate(image, angle)  
    aug_images.append(rotated_image)
    return aug_images

def tta_predict(model, image, device):
    augmented_images = tta_augmentations(image)
    weights = torch.ones(len(augmented_images), dtype=torch.float32, device=device) / len(augmented_images)

    final_prediction = None
    model.eval()
    with torch.no_grad():
        for i, aug_image in enumerate(augmented_images):
            aug_image = aug_image.unsqueeze(0).to(device)
            prediction = model(aug_image)
            if final_prediction is None:
                final_prediction = weights[i] * prediction
            else:
                final_prediction += weights[i] * prediction

    return final_prediction

In [22]:
# Test
SUBMISSION_ROWS = []
model.eval()

for X_sample_test, test_id in tqdm(test_dataset):
    with torch.no_grad():
        prediction = tta_predict(model, X_sample_test, 'cuda').detach().cpu().numpy().squeeze()

    y_pred = SCALER.inverse_transform([prediction])[0]
    row = {'id': test_id}

    for k, v in zip(CONFIG.TARGET_COLUMNS, y_pred):
        if k in LOG_FEATURES:
            row[k.replace('_mean', '')] = 10 ** v
        else:
            row[k.replace('_mean', '')] = v

    SUBMISSION_ROWS.append(row)

submission_eda = pd.DataFrame(SUBMISSION_ROWS).copy()

submission_eda

  0%|          | 0/6391 [00:00<?, ?it/s]

TypeError: 'collections.OrderedDict' object is not callable

In [26]:
import numpy as np
import pandas as pd
import imageio.v3 as imageio
import albumentations as A

# import torch_xla as xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.distributed.xla_backend

from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch import nn
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform


import torch
import timm
import torchmetrics
import time
import psutil

In [27]:
class Config():
    IMAGE_SIZE = 384
    BACKBONE = 'swin_large_patch4_window12_384.ms_in22k_ft_in1k'
    TARGET_COLUMNS = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
    N_TARGETS = len(TARGET_COLUMNS)
    BATCH_SIZE = 64
    LR_MAX = 1e-4
    WEIGHT_DECAY = 0.01
    N_EPOCHS = 5
    TRAIN_MODEL = True
    IS_INTERACTIVE = True  # os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
    tpu_ids = range(8)
    Lower_Quantile = 0.005
    Upper_Quantile = 0.985
    SHRINK_SAMPLES = True

CONFIG = Config()

class TrainDataset(Dataset):
    def __init__(self, X_jpeg_bytes, X_tabular, y, transforms=None):
        self.X_jpeg_bytes = X_jpeg_bytes
        self.X_tabular = X_tabular
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        X_sample = self.transforms(
            image=imageio.imread(self.X_jpeg_bytes[index]),
        )['image']
        X_tabular_sample = self.X_tabular[index]
        y_sample = self.y[index]

        return X_sample, X_tabular_sample, y_sample
    
    
class TabularBackbone(nn.Module):
    def __init__(self, n_features, out_features):
        super().__init__()
        self.out_features = out_features
        self.fc = nn.Sequential(
            nn.Linear(n_features, 512),
            
            nn.BatchNorm1d(512),
            nn.GELU(),
            # nn.Dropout(0.1),
            nn.Linear(512, out_features),
        )

    def forward(self, x):
        return self.fc(x)
    
class ImageBackbone(nn.Module):
    def __init__(self, backbone_name, weight_path, out_features, fixed_feature_extractor=False):
        super().__init__()
        self.out_features = out_features
        self.backbone = timm.create_model(backbone_name, pretrained=False, num_classes=CONFIG.N_TARGETS)
        self.backbone.load_state_dict(torch.load(weight_path))
        if fixed_feature_extractor:
            for param in self.backbone.parameters():
                param.requires_grad = False
        in_features = self.backbone.num_features
        
        self.backbone.head = nn.Identity()
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(in_features, out_features),
        )

    def forward(self, x):
        x = self.backbone(x)
        x = x.permute(0, 3, 1, 2)
        return self.head(x)

class Model(nn.Module):
    def __init__(self, img_backbone, tab_backbone, out_features:int):
        super().__init__()
        self.img_backbone = img_backbone
        self.tab_backbone = tab_backbone
        self.fc = nn.Sequential(
            nn.Linear(self.tab_backbone.out_features + self.img_backbone.out_features, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            # nn.Dropout(0.1),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            # nn.Dropout(0.1),
            nn.Linear(256, out_features),
        )

    def forward(self, img, tab):
        img_features = self.img_backbone(img)
        tab_features = self.tab_backbone(tab)
        features = torch.cat([img_features, tab_features], dim=1)
        return self.fc(features)
        

In [29]:
img_backbone = ImageBackbone('swin_large_patch4_window12_384.ms_in22k_ft_in1k', f'{BASE_DIR}/model_08_ensemble.pth', 384, fixed_feature_extractor=True)
tab_backbone = TabularBackbone(n_features=tabular_scaled.shape[1], out_features=128)

model = Model(img_backbone, tab_backbone, CONFIG.N_TARGETS)
model = model.to(DEVICE)
state_dict = torch.load(f'{BASE_DIR}/model_08_ensemble.pth')
model.load_state_dict(state_dict)

/tmp/ipykernel_79770/3158310507.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.backbone.load_state_dict(torch.load(weight_path))


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/swin-transformer-v1-planttraits2024-finetuned/pytorch/log3-noval-8epoch/1/model_08.pth'

In [31]:
import kagglehub

# Download latest version
path = kagglehub.model_download("riverfog7/swin-transformer-v1-planttraits2024-finetuned/pyTorch/log3-noval-8epoch")

100%|██████████| 745M/745M [00:14<00:00, 52.1MB/s]


In [32]:
path

'/home/prajwal/.cache/kagglehub/models/riverfog7/swin-transformer-v1-planttraits2024-finetuned/pyTorch/log3-noval-8epoch/1'